In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings 
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans

In [2]:
df = pd.read_pickle('data/latest_merged.pkl')

In [3]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",100)
pd.set_option("display.max_colwidth",None)

In [5]:
df.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,No_of_PreviousApplications,Credit_to_Income_Ratio,Annuity_to_Income_Ratio,GoodsPrice_to_Income_Ratio,No_of_DocumentsSubmitted,EXT_SCORES_AVG,EXT_SCORES_SUM,FLAG_CONTACT_INFO,EXT_SOURCE_1,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Other,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Security staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_XNA,clusters,AMT_ANNUITY_y_list,AMT_APPLICATION_list,AMT_CREDIT_y_list,AMT_DOWN_PAYMENT_list,AMT_GOODS_PRICE_y_list,HOUR_APPR_PROCESS_START_y_list,NFLAG_LAST_APPL_IN_DAY_list,RATE_DOWN_PAYMENT_list,DAYS_DECISION_list,SELLERPLACE_AREA_list,CNT_PAYMENT_list,DAYS_FIRST_DRAWING_list,DAYS_FIRST_DUE_list,DAYS_LAST_DUE_1ST_VERSION_list,DAYS_LAST_DUE_list,DAYS_TERMINATION_list,NFLAG_INSURED_ON_APPROVAL_list,NAME_CONTRACT_TYPE_y_list_Cash loans,NAME_CONTRACT_TYPE_y_list_Consumer loans,NAME_CONTRACT_TYPE_y_list_Revolving loans,NAME_CONTRACT_TYPE_y_list_XNA,WEEKDAY_APPR_PROCESS_START_y_list_FRIDAY,WEEKDAY_APPR_PROCESS_START_y_list_MONDAY,WEEKDAY_APPR_PROCESS_START_y_list_SATURDAY,WEEKDAY_APPR_PROCESS

In [4]:
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,RandomizedSearchCV,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score,confusion_matrix,r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from xgboost import XGBClassifier
import lightgbm as lgm

In [5]:
def split(dataframe):
    X = dataframe.drop('TARGET',axis=1)
    y = dataframe['TARGET']
    y = y.astype('int')

    X_train1,X_test,y_train1,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

    X_train,X_val,y_train,y_val = train_test_split(X_train1,y_train1,test_size=0.3,random_state=0)
    return X_train1,X_test,y_train1,y_test,X_train,X_val,y_train,y_val

def fit_predict(model):
    model.fit(X_train,y_train)

    y_pred = model.predict(X_val)

    print("Accuracy = {0} \nRecall {1} \nPrecision {2} \nF1 score {3}".format( 
        accuracy_score(y_val,y_pred),recall_score(y_val,y_pred),
        precision_score(y_val,y_pred),f1_score(y_val,y_pred)))

def cross_validate(model,scoring_metric,folds):  
    skf = StratifiedKFold(folds,shuffle=True,random_state=0)
    scores = cross_val_score(log,X_train1,y_train1,cv=skf,scoring=scoring_metric)

    print(scores)
    print(np.mean(scores))

def feature_importance(model):

    feature_imp =  pd.DataFrame(model.feature_importances_,index=list(df.columns.drop('TARGET')),
              columns=['feature_imp']).sort_values('feature_imp',ascending=False)
    imp_columns = list(feature_imp[feature_imp['feature_imp']>0].index)
    imp_df = df[imp_columns]
    imp_df['TARGET'] = df['TARGET']
    return feature_imp,imp_df

In [31]:
X_train1,X_test,y_train1,y_test,X_train,X_val,y_train,y_val = split(df)

In [9]:
X = df.drop('TARGET',axis=1)
y = df['TARGET']
y = y.astype('int')

In [10]:
# skf = StratifiedKFold(10,shuffle=True,random_state=0)
# i = 1
# splits = {}
# for train_index,test_index in skf.split(X,y):
# #     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X.iloc[list(train_index)], X.iloc[list(test_index)]
#     y_train, y_test = y.iloc[list(train_index)], y.iloc[list(test_index)]
#     xtrain_key = "X_train"+str(i)
#     ytrain_key = "y_train"+str(i)
#     xtest_key = "X_test"+str(i)
#     ytest_key = "y_test"+str(i)
#     splits[xtrain_key] = X_train
#     splits[xtest_key] = X_test
#     splits[ytrain_key] = y_train
#     splits[ytest_key] = y_test
#     i = i+1

In [11]:
# splits.keys()

dict_keys(['X_train1', 'X_test1', 'y_train1', 'y_test1', 'X_train2', 'X_test2', 'y_train2', 'y_test2', 'X_train3', 'X_test3', 'y_train3', 'y_test3', 'X_train4', 'X_test4', 'y_train4', 'y_test4', 'X_train5', 'X_test5', 'y_train5', 'y_test5', 'X_train6', 'X_test6', 'y_train6', 'y_test6', 'X_train7', 'X_test7', 'y_train7', 'y_test7', 'X_train8', 'X_test8', 'y_train8', 'y_test8', 'X_train9', 'X_test9', 'y_train9', 'y_test9', 'X_train10', 'X_test10', 'y_train10', 'y_test10'])

In [7]:
int(np.round(len(y_val)))

40447

In [8]:
length = int(np.round(len(y_val)/10))
splits = {}
i = 1
j = 0
while i<=10:
    xtrain_key = "X_train"+str(i)
    ytrain_key = "y_train"+str(i)
    xtest_key = "X_test"+str(i)
    ytest_key = "y_test"+str(i)
    splits[xtrain_key] = X_train[j:j+length]
    splits[xtest_key] = X_test[j:j+length]
    splits[ytrain_key] = y_train[j:j+length]
    splits[ytest_key] = y_test[j:j+length]
    i = i+1
    j = j+length

In [9]:
splits.keys()

dict_keys(['X_train1', 'X_test1', 'y_train1', 'y_test1', 'X_train2', 'X_test2', 'y_train2', 'y_test2', 'X_train3', 'X_test3', 'y_train3', 'y_test3', 'X_train4', 'X_test4', 'y_train4', 'y_test4', 'X_train5', 'X_test5', 'y_train5', 'y_test5', 'X_train6', 'X_test6', 'y_train6', 'y_test6', 'X_train7', 'X_test7', 'y_train7', 'y_test7', 'X_train8', 'X_test8', 'y_train8', 'y_test8', 'X_train9', 'X_test9', 'y_train9', 'y_test9', 'X_train10', 'X_test10', 'y_train10', 'y_test10'])

### Logistic Regression

In [10]:
splits['y_train1'].value_counts(),splits['y_test1'].value_counts()

(0    3726
 1     319
 Name: TARGET, dtype: int64,
 0    3708
 1     337
 Name: TARGET, dtype: int64)

In [44]:
def ks_model(split_no):
    xtrain_key = "X_train"+str(split_no)
    ytrain_key = "y_train"+str(split_no)
    xtest_key = "X_test"+str(split_no)
    ytest_key = "y_test"+str(split_no)
    log = LogisticRegression(random_state=0)
    log.fit(splits[xtrain_key],splits[ytrain_key])
    y_pred = log.predict(splits[xtest_key])
    return y_pred
y_pred = ks_model(10)

In [45]:
key = 'y_test10'
log = LogisticRegression(random_state=0)
y_pred = pd.Series(y_pred,index=list(splits[key].index))
accuracy = accuracy_score(splits[key],y_pred)
skf = StratifiedKFold(10,shuffle=True,random_state=0)
scores = cross_val_score(log,splits['X_train10'],splits['y_train10'],cv=skf,scoring='accuracy')

print(scores)
print(np.mean(scores))

splits[key].value_counts(),y_pred.value_counts(),accuracy

[0.90864198 0.9037037  0.91358025 0.91358025 0.8962963  0.91336634
 0.91089109 0.90841584 0.90594059 0.91089109]
0.9085307419630853


(0    3699
 1     346
 Name: TARGET, dtype: int64,
 0    3996
 1      49
 dtype: int64,
 0.9107540173053152)

In [46]:
scores = list(scores)
scores.sort()
scores

[0.8962962962962963,
 0.9037037037037037,
 0.905940594059406,
 0.9084158415841584,
 0.908641975308642,
 0.9108910891089109,
 0.9108910891089109,
 0.9133663366336634,
 0.9135802469135802,
 0.9135802469135802]

### KS for train 

In [7]:
log = LogisticRegression(random_state=42)
# fit_predict(log)

In [8]:
log.fit(X_train,y_train)
y_pred = log.predict(X_val)
accuracy_score(y_val,y_pred)

0.9172747988116184

In [9]:
pred_proba =  log.predict_proba(X_val)

In [10]:
X_val['proba'] = pd.DataFrame(pred_proba,index=list(y_val.index),columns=['proba',1])['proba']
X_val['TARGET'] = y_val
X_val['TARGET0'] = 1-X_val['TARGET']
X_val['bucket'] = pd.qcut(X_val['proba'],10)

In [235]:
grouped = X_val.groupby('bucket',as_index=False)

In [221]:
grouped.min()['proba']

0    0.162211
1    0.821568
2    0.879078
3    0.909290
4    0.929640
5    0.944618
6    0.956327
7    0.966059
8    0.974639
9    0.982992
Name: proba, dtype: float64

In [222]:
ks_df = pd.DataFrame()
ks_df['Model_score_starting'] = grouped.min().proba
ks_df['Model_score_ending'] = grouped.max().proba
ks_df['no of good customers'] = grouped.sum().TARGET0
ks_df['no of bad customers'] = grouped.sum().TARGET


In [223]:
ks_df

,Model_score_starting,Model_score_ending,no of good customers,no of bad customers
0,0.162211,0.821561,5047,1887
1,0.821568,0.879071,5898,1036
2,0.879078,0.909290,6212,722
3,0.909290,0.929635,6340,593
4,0.929640,0.944617,6509,425
5,0.944618,0.956324,6574,360
6,0.956327,0.966056,6636,297
7,0.966059,0.974638,6756,178
8,0.974639,0.982990,6789,145
9,0.982992,1.000000,6830,104


In [224]:
ks_df.to_csv('KS/KS_train_df.csv')

In [225]:
len(y_train),len(y_val),len(y_test)

(161787, 69338, 57782)

In [196]:
len(df)

288907

In [212]:
57782/10

5778.2

### Ks for Test

In [30]:
list(X_val['bucket'].value_counts().index)

[Interval(0.983, 1.0, closed='right'),
 Interval(0.975, 0.983, closed='right'),
 Interval(0.966, 0.975, closed='right'),
 Interval(0.945, 0.956, closed='right'),
 Interval(0.93, 0.945, closed='right'),
 Interval(0.879, 0.909, closed='right'),
 Interval(0.822, 0.879, closed='right'),
 Interval(0.161, 0.822, closed='right'),
 Interval(0.956, 0.966, closed='right'),
 Interval(0.909, 0.93, closed='right')]

In [32]:
pred_proba =  log.predict_proba(X_test)

In [34]:
X_test['proba'] = pd.DataFrame(pred_proba,index=list(y_test.index),columns=['proba',1])['proba']
X_test['TARGET'] = y_test
X_test['TARGET0'] = 1-X_test['TARGET']
proba_bins = [0.161,0.822,0.879,0.909,0.93,0.945,0.956,0.966,0.975,0.983,1]
X_test['bucket'] = pd.cut(X_test['proba'],bins=proba_bins)

In [39]:
grouped = X_test.groupby('bucket',as_index=False)

In [40]:
grouped.min()['proba']

0    0.291437
1    0.822000
2    0.879005
3    0.909000
4    0.930000
5    0.945005
6    0.956002
7    0.966001
8    0.975002
9    0.983001
Name: proba, dtype: float64

In [41]:
ks_df = pd.DataFrame()
ks_df['Model_score_starting'] = grouped.min().proba
ks_df['Model_score_ending'] = grouped.max().proba
ks_df['no of good customers'] = grouped.sum().TARGET0
ks_df['no of bad customers'] = grouped.sum().TARGET


In [42]:
ks_df

,Model_score_starting,Model_score_ending,no of good customers,no of bad customers
0,0.291437,0.821983,4472,1546
1,0.822000,0.878999,5058,927
2,0.879005,0.908998,5111,604
3,0.909000,0.929997,5415,473
4,0.930000,0.944998,5399,341
5,0.945005,0.955999,5132,257
6,0.956002,0.965999,5620,219
7,0.966001,0.974999,5637,167
8,0.975002,0.982998,5375,123
9,0.983001,0.998695,5820,86


In [43]:
ks_df.to_csv('KS/KS_test_df.csv')

In [180]:
len(X_test)/10

8667.3

In [14]:
gb = GradientBoostingClassifier(random_state=0)

fit_predict(gb)

Accuracy = 0.9193018023586422 
Recall 0.017927681555758128 
Precision 0.6483516483516484 
F1 score 0.03489059727971615
-0.07964164591027356


####  KS

In [83]:
from scipy.stats import kstest,ks_2samp
import scipy.stats as st

In [68]:
ks_df = pd.DataFrame(y_pred,index=list(y_val.index),columns=['y_pred'])
ks_df['y_val'] = y_val
ks_df

,y_pred,y_val
162042,0,1
265018,0,0
113477,0,0
404044,0,1
281816,0,0
...,...,...
314246,0,0
405395,0,0
322190,0,0
198493,0,1


In [78]:
st.norm.cdf(y_pred)

array([0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5])

In [82]:
st.percentileofscore([1,2,3,3,4],3)

70.0

In [76]:
ks_df['y_pred'].min()

0

In [70]:
kstest(y_val,y_pred)

KstestResult(statistic=0.07921477489059758, pvalue=7.880954661177253e-111)

In [128]:
ks_2samp(y_val,y_pred)

KstestResult(statistic=0.07921477489059758, pvalue=7.880954661177253e-111)